## Snake plot
Three- or four-element plots (we call them _snake_ plots) are especially useful to compare the predictions of different neutron density solutions with elemental ratios.  This is a plot where we have for each neutron density one dot in a three- or four-element plot.

This is the _short_ version of the notebook that just plots precomputed one-zone equilibrium solutions with pre-selected JINABase observations of CEMP stars. There is a _long_ version that allows you to add additional model files that you may have created.

In [9]:
%pylab ipympl
from nugridpy import utils
from nugridpy import ppn
import pickle
import sys, operator, glob, os

Populating the interactive namespace from numpy and matplotlib


In [10]:
data_dir = '/data/nugrid_data/projects/tina/iprocess-impact-MC/'
casename = 'default'

In [11]:
import jbutils # this is a local module in this directyory

# data_dir needs to be the same in `jbutils.py` as above!

In [12]:
data_dir+'data_utils/Nn_elem_dict_'+casename+'.pkl'

'/data/nugrid_data/projects/tina/iprocess-impact-MC/data_utils/Nn_elem_dict_default.pkl'

In [13]:
# read Nn_elem_dict.pkl for plotting
# this file contains some precalculated constant density runs
afile = open(data_dir+'data_utils/Nn_elem_dict_'+casename+'.pkl','rb') 
Nn,elem_dict = pickle.load(afile)
afile.close()

# in the long version you can create these files for different assumptions
# and add them to the snake plot

In [14]:
# set elements to plot: 
# [els[0], els[1], els[2], els[3]] 
# [els[0]/els[1]] on x axis [els[2]/els[3]] on y axis
els = ['Ba', 'La', 'La','Eu']
# els = ['Ce', 'Nd', 'Sm','Eu']

# to set Nn colorbar range find 
logNn = log10(array(list(Nn.values())))
min_,max_ = logNn.min(),logNn.max()

size = 6
close(74);figure(74,figsize=(1.6*size,size))
scatter(elem_dict[els[0]]-elem_dict[els[1]],elem_dict[els[2]]-elem_dict[els[3]],\
        c=logNn,cmap='jet')
plot(elem_dict[els[0]]-elem_dict[els[1]],elem_dict[els[2]]-elem_dict[els[3]],'--',\
     color='grey',label='default')


jbutils.plot_elemratios(els=els,selections = ['S', 'I', 'R1', 'R2'], uls=False)        

clim(min_, max_) # set the min/max of the colorbar 
cbar = colorbar()
cbar.set_label('$\mathsf{N_n}$', rotation=90)


# legend()
xlim(-0.6,1.),ylim(-.6,1.3)
xlabel('['+els[0]+'/'+els[1]+']');ylabel('['+els[2]+'/'+els[3]+']')
tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Add a point from a new run

In [16]:
ncases = ['Ba139up50_1.00d+14'] # this is a list of your runs you made in ../i-process-Nnconst

In [17]:
# %%capture
# create dictionary only first time
# to add new cases comment
pa_cases = {} 

# create a ppn.abu_vector instance for each case
for case in ncases:
    pa=ppn.abu_vector(os.path.join("/home/user/Network-school/abc/i-process-Nnconst",case))
    pa_cases[case] = pa

524 cycle numbers found in /home/user/Network-school/abc/i-process-Nnconst/Ba139up50_1.00d+14
Ranging from 00000 to 00523
Range may not be continuous. To display all available cycles, print <abu_vector_instance>.files


Run the next cell to exstract the elemental abundance and Nn dictionaries you need to add to the snake plot.

In [18]:
%%capture
el_abu_log_dict={}; Nn={} 
data_dir = '/data/nugrid_data/projects/tina/iprocess-impact-MC/'
sol_ab = data_dir+'one-zone-run_default/'+'../data_utils/iniab1.4E-02As09.ppn'
for i,case in enumerate(ncases):
    close(100+i);figure(100+i)
    print (i,case)
    cycle = pa_cases[case].get('mod')[-1]                                                 # this is the last cycle
    pa_cases[case].elemental_abund(cycle,zrange=[25,85],ref_filename=sol_ab,ylim=[-2,8])  # elements only returned that are in zrange
    el_abu_log_dict[case] = pa_cases[case].el_abu_log
    Nn[case]              = pa_cases[case].get('densn',cycle)
    title(case)


The following cell will make sure your results are sorted by Nn so that a connected additional snake line is plotted. Also you will save this snake in a `.pkl` file.

In [19]:
casename = 'Ba139up50'
# sort, create elem_dict and write if sort_Nn = True
sort_Nn = True 
if sort_Nn:
    Nnsorted = sorted(Nn.items(), key=operator.itemgetter(1))
    cases_sorted = [Nnsorted[i][0] for i in range(len(Nnsorted))]
    Nnsorted = {}
    for case in cases_sorted: Nnsorted[case] = Nn[case]
    #print(cases_sorted)

    # create a elemental abundance dictionary in which for each available element 
    # the log elem abundance as extracted during elemental_abund plotting are available

    elem_dict_add = {} #always start with a new dictionary

    for element in pa.el_name:
        elem_dict_add[element] = array([el_abu_log_dict[case][pa.el_name.index(element)] for case in cases_sorted])

    # write elem_dict dictionary into python pickle
    afile = open('Nn_elem_dict_'+casename+'.pkl', 'wb')
    pickle.dump([Nnsorted,elem_dict], afile)
    afile.close()

In [20]:
figure(74)
plot(elem_dict_add[els[0]]-elem_dict_add[els[1]],elem_dict_add[els[2]]-elem_dict_add[els[3]],'sb')
